In [2]:

# =====================================================================
# SMC Swing Trading Backtest (Python, TA‑Lib, optional 'smartmoneyconcepts')
# =====================================================================
# - Downloads OHLC data for a LIST of tickers
# - Computes TA-Lib indicators (SMA/RSI/ATR)
# - Uses `smartmoneyconcepts` (if installed) for SMC features (FVG, BOS/CHOCH, OB);
#   otherwise falls back to a simple FVG + BOS proxy
# - Generates entry/exit signals and runs a pure‑Python backtest
# - Saves a summary CSV & trades CSV
#
# Usage:
#   1) pip install yfinance pandas numpy matplotlib ta-lib smartmoneyconcepts
#      - If TA-Lib fails on macOS: brew install ta-lib
#   2) Edit the PARAMS section below
#   3) python smc_backtest.py
# =====================================================================
# !pip install smartmoneyconcepts
import sys, math, warnings, datetime as dt
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# -----------------------
# 0) Third‑party packages
# -----------------------
try:
    import yfinance as yf
except Exception as e:
    raise SystemExit("yfinance is required. Install: pip install yfinance") from e

try:
    import talib as ta
    HAVE_TALIB = True
except Exception as e:
    HAVE_TALIB = False
    print("[WARN] TA-Lib not found – will use pandas fallbacks for indicators.", e)

HAVE_SMC_LIB = False
try:
    import smartmoneyconcepts as smc
    HAVE_SMC_LIB = True
except Exception as e:
    print("[INFO] `smartmoneyconcepts` not found – using fallback SMC detectors (FVG + basic BOS).", e)

# -----------------------
# 1) PARAMS (EDIT ME)
# -----------------------
TICKERS      = ["RELIANCE.NS", "TCS.NS", "INFY.NS", "HDFCBANK.NS"]
START_DATE   = "2018-01-01"
END_DATE     = None          # None -> today
INTERVAL     = "1d"          # "1d", "1h", "15m" (Yahoo intraday has history limits)

# Signal & filter params
USE_SMA_TREND   = True
SMA_FAST        = 50
SMA_SLOW        = 200

USE_RSI_FILTER  = True
RSI_LEN         = 14
RSI_MIN_LONG    = 40
RSI_MAX_SHORT   = 60

# Risk & exits
ATR_LEN         = 14
ATR_MULT_SL     = 1.5        # stop = entry - ATR_MULT_SL * ATR (longs)
TP_R_MULT       = 2.0        # take-profit = entry + TP_R_MULT * (entry - stop)
RISK_PCT        = 0.01       # risk per trade (1%) for position sizing
SLIPPAGE_PCT    = 0.0005     # 5 bps slippage each side
FEE_PCT         = 0.00025    # 2.5 bps fees each side

# SMC signal logic
REQUIRE_BULLISH_BOS_FOR_LONG = True
REQUIRE_BEARISH_BOS_FOR_SHORT= False   # enable if you use shorts
USE_ORDER_BLOCKS             = True    # only effective when HAVE_SMC_LIB
USE_FVG_ZONES                = True

# Backtest
ALLOW_SHORTS    = False       # set True to enable shorting
CAPITAL_PER_TKR = 200_000.0
TP_HIT_PRIORITY = "stop_first"  # "stop_first" or "tp_first" if both hit same bar

RANDOM_SEED     = 42
pd.options.display.float_format = "{:,.4f}".format

# -----------------------
# 2) UTILS & INDICATORS
# -----------------------
def load_ohlc(ticker, start=START_DATE, end=END_DATE, interval=INTERVAL):
    end = end or dt.date.today().isoformat()
    df = yf.download(ticker, start=start, end=end, interval=interval, auto_adjust=False, progress=False, multi_level_index=False)
    if df.empty:
        raise ValueError(f"No data for {ticker} at {interval}.")
    df = df.rename(columns=str.title).dropna()
    return df

def sma(series, length):
    if HAVE_TALIB:
        return pd.Series(ta.SMA(series.values.astype(float), timeperiod=length), index=series.index)
    return series.rolling(length, min_periods=1).mean()

def rsi(series, length=14):
    if HAVE_TALIB:
        return pd.Series(ta.RSI(series.values.astype(float), timeperiod=length), index=series.index)
    delta = series.diff()
    gain = (delta.clip(lower=0)).ewm(alpha=1/length, adjust=False).mean()
    loss = (-delta.clip(upper=0)).ewm(alpha=1/length, adjust=False).mean()
    rs = gain / (loss.replace(0, np.nan))
    return 100 - (100 / (1 + rs))

def atr(df, length=14):
    high, low, close = df["High"], df["Low"], df["Close"]
    if HAVE_TALIB:
        return pd.Series(ta.ATR(high.values.astype(float), low.values.astype(float), close.values.astype(float), timeperiod=length), index=df.index)
    h_l   = high - low
    h_pc  = (high - close.shift()).abs()
    l_pc  = (low - close.shift()).abs()
    tr = pd.concat([h_l, h_pc, l_pc], axis=1).max(axis=1)
    return tr.rolling(length, min_periods=1).mean()

def add_indicators(df):
    df["SMA_FAST"] = sma(df["Close"], SMA_FAST)
    df["SMA_SLOW"] = sma(df["Close"], SMA_SLOW)
    df["RSI"]      = rsi(df["Close"], RSI_LEN)
    df["ATR"]      = atr(df, ATR_LEN)
    return df

# -----------------------
# 3) SMC PRIMITIVES
# -----------------------
def detect_smc_with_library(df):
    """
    Use `smartmoneyconcepts` to compute SMC features.
    Returns a dict of DataFrames (joined later).
    """
    ohlc = df[["Open","High","Low","Close"]].copy()
    out = {}
    try:
        out["fvg"] = smc.fvg(ohlc)
    except Exception as e:
        print("smc.fvg failed:", e); out["fvg"] = pd.DataFrame(index=df.index)
    try:
        swings = smc.highs_lows(ohlc)
    except Exception as e:
        print("smc.highs_lows failed:", e); swings = pd.DataFrame(index=df.index)
    try:
        out["bos"] = smc.bos_choch(ohlc, close_break=True)
    except Exception as e:
        print("smc.bos_choch failed:", e); out["bos"] = pd.DataFrame(index=df.index)
    try:
        out["ob"] = smc.ob(ohlc)
    except Exception as e:
        print("smc.ob failed:", e); out["ob"] = pd.DataFrame(index=df.index)
    try:
        out["liq"] = smc.liquidity(ohlc)
    except Exception:
        out["liq"] = pd.DataFrame(index=df.index)
    return out

def detect_fvg_fallback(df):
    """
    Simple Fair Value Gap detector (3-candle logic).
      - Bullish FVG: low[i+1] > high[i-1] -> zone [high[i-1], low[i+1]]
      - Bearish FVG: high[i+1] < low[i-1] -> zone [high[i+1], low[i-1]]
    """
    H, L = df["High"].values, df["Low"].values
    up_start = np.full(len(df), np.nan); up_end = np.full(len(df), np.nan)
    dn_start = np.full(len(df), np.nan); dn_end = np.full(len(df), np.nan)
    for i in range(1, len(df)-1):
        if L[i+1] > H[i-1]:          # bullish gap
            up_start[i] = H[i-1]
            up_end[i]   = L[i+1]
        if H[i+1] < L[i-1]:          # bearish gap
            dn_start[i] = H[i+1]
            dn_end[i]   = L[i-1]
    return pd.DataFrame({
        "fvg_up_start": up_start, "fvg_up_end": up_end,
        "fvg_dn_start": dn_start, "fvg_dn_end": dn_end
    }, index=df.index)

def detect_basic_bos(df, lookback=10):
    """
    Basic BOS proxy:
      - bullish if close > max(High[-lookback:-1])
      - bearish if close < min(Low[-lookback:-1])
    """
    highs_roll = df["High"].shift(1).rolling(lookback, min_periods=1).max()
    lows_roll  = df["Low"].shift(1).rolling(lookback, min_periods=1).min()
    bos_bull = df["Close"] > highs_roll
    bos_bear = df["Close"] < lows_roll
    return pd.DataFrame({"bos_bull": bos_bull, "bos_bear": bos_bear}, index=df.index)

def add_smc(df):
    if HAVE_SMC_LIB:
        smc_all = detect_smc_with_library(df)
        for k, v in smc_all.items():
            if v is not None and not v.empty:
                df = df.join(v.add_prefix(f"smc_{k}_"))
        # If no BOS columns were provided, add fallback
        if not any("bos" in c.lower() for c in df.columns):
            df = df.join(detect_basic_bos(df, lookback=10))
    else:
        df = df.join(detect_fvg_fallback(df))
        df = df.join(detect_basic_bos(df, lookback=10))
    return df

# -----------------------
# 4) SIGNALS
# -----------------------
def _pick_recent_zone(row_idx, df, kind="bullish"):
    if HAVE_SMC_LIB:
        # Try to find any FVG-like pair in smc-joined columns
        candidates = []
        for a,b in [
            ("smc_fvg_bullish_fvg_start", "smc_fvg_bullish_fvg_end"),
            ("smc_fvg_fvg_up_start", "smc_fvg_fvg_up_end"),
            ("fvg_up_start", "fvg_up_end"),
            ("smc_fvg_bearish_fvg_start", "smc_fvg_bearish_fvg_end"),
            ("smc_fvg_fvg_dn_start", "smc_fvg_fvg_dn_end"),
            ("fvg_dn_start", "fvg_dn_end"),
        ]:
            if (a in df.columns) and (b in df.columns):
                candidates.append((a,b))
        if not candidates:
            return (np.nan, np.nan)
        # pick most recent non-NaN row <= row_idx
        for a,b in candidates:
            sub = df.loc[:df.index[row_idx], [a,b]].dropna()
            if not sub.empty:
                s,e = sub.iloc[-1]
                return float(s), float(e)
        return (np.nan, np.nan)
    else:
        if kind == "bullish":
            sub = df.loc[:df.index[row_idx], ["fvg_up_start","fvg_up_end"]].dropna()
        else:
            sub = df.loc[:df.index[row_idx], ["fvg_dn_start","fvg_dn_end"]].dropna()
        if sub.empty: return (np.nan, np.nan)
        s,e = sub.iloc[-1]
        return float(s), float(e)

def _in_zone(price, start, end):
    if any(map(lambda x: np.isnan(x), [price, start, end])):
        return False
    lo, hi = (start, end) if start <= end else (end, start)
    return (price >= lo) and (price <= hi)

def gen_signals(df):
    d = df.copy()
    trend_up   = (d["SMA_FAST"] > d["SMA_SLOW"]) if USE_SMA_TREND else pd.Series(True, index=d.index)
    trend_down = (d["SMA_FAST"] < d["SMA_SLOW"]) if USE_SMA_TREND else pd.Series(True, index=d.index)
    rsi_ok_long  = (d["RSI"] >= RSI_MIN_LONG) if USE_RSI_FILTER else pd.Series(True, index=d.index)
    rsi_ok_short = (d["RSI"] <= RSI_MAX_SHORT) if USE_RSI_FILTER else pd.Series(True, index=d.index)

    # BOS flags (heuristic for lib, fallback otherwise)
    if HAVE_SMC_LIB and any(("bos" in c.lower()) for c in d.columns):
        bos_cols = [c for c in d.columns if "bos" in c.lower()]
        bos_bull = pd.Series(False, index=d.index)
        bos_bear = pd.Series(False, index=d.index)
        for c in bos_cols:
            vals = d[c].astype(str)
            bos_bull = bos_bull | vals.str.contains("BULL", case=False, na=False)
            bos_bear = bos_bear | vals.str.contains("BEAR", case=False, na=False)
    else:
        bos_bull = d.get("bos_bull", pd.Series(False, index=d.index))
        bos_bear = d.get("bos_bear", pd.Series(False, index=d.index))

    long_entry = []
    short_entry = []
    closes = d["Close"].values

    for i in range(len(d)):
        c = closes[i]
        bull_s, bull_e = _pick_recent_zone(i, d, kind="bullish") if USE_FVG_ZONES else (np.nan, np.nan)
        bear_s, bear_e = _pick_recent_zone(i, d, kind="bearish") if (USE_FVG_ZONES and ALLOW_SHORTS) else (np.nan, np.nan)
        tap_bull = _in_zone(c, bull_s, bull_e) if USE_FVG_ZONES else True
        tap_bear = _in_zone(c, bear_s, bear_e) if (USE_FVG_ZONES and ALLOW_SHORTS) else True

        # OB filters could be enforced here if lib columns are known;
        # for generality, we keep them permissive by default.
        ob_ok_long = True
        ob_ok_short = True

        le = (trend_up.iloc[i] and rsi_ok_long.iloc[i] and tap_bull and ob_ok_long and
              ((not REQUIRE_BULLISH_BOS_FOR_LONG) or bos_bull.iloc[i]))
        se = False
        if ALLOW_SHORTS:
            se = (trend_down.iloc[i] and rsi_ok_short.iloc[i] and tap_bear and ob_ok_short and
                  ((not REQUIRE_BEARISH_BOS_FOR_SHORT) or bos_bear.iloc[i]))

        long_entry.append(bool(le))
        short_entry.append(bool(se))

    d["long_entry"]  = long_entry
    d["short_entry"] = short_entry
    d["R_per_price"] = d["ATR"]
    return d

# -----------------------
# 5) BACKTEST
# -----------------------
from dataclasses import dataclass

@dataclass
class Trade:
    side: str
    entry_time: pd.Timestamp
    entry_px: float
    stop_px: float
    tp_px: float
    size: float
    exit_time: pd.Timestamp = None
    exit_px: float = None
    pnl: float = None
    r_mult: float = None
    entry_px_after_costs: float = None

def _size_position(capital, risk_pct, entry_px, stop_px):
    risk_per_share = abs(entry_px - stop_px)
    if risk_per_share <= 0: return 0.0
    cash_at_risk = capital * risk_pct
    shares = math.floor(cash_at_risk / risk_per_share)
    return max(shares, 0)

def _apply_costs(price):
    # returns (buy_costed, sell_costed) around given 'price'
    return price * (1 + SLIPPAGE_PCT + FEE_PCT), price * (1 - SLIPPAGE_PCT - FEE_PCT)

def simulate_ticker(df):
    equity = CAPITAL_PER_TKR
    open_pos = None
    trades = []

    for i in range(len(df)-1):
        t, nt = df.index[i], df.index[i+1]
        row = df.iloc[i]
        nx  = df.iloc[i+1]

        # Manage open trade -> exits on next bar
        if open_pos is not None:
            if open_pos.side == "long":
                hit_sl = (nx["Low"]  <= open_pos.stop_px)
                hit_tp = (nx["High"] >= open_pos.tp_px)
                exit_px = None
                when = nt
                if hit_sl and hit_tp:
                    exit_px = open_pos.tp_px if TP_HIT_PRIORITY == "tp_first" else open_pos.stop_px
                elif hit_sl: exit_px = open_pos.stop_px
                elif hit_tp: exit_px = open_pos.tp_px
                if exit_px is not None:
                    # selling out of long
                    _, exit_after_costs = _apply_costs(exit_px)
                    pnl = (exit_after_costs - open_pos.entry_px_after_costs) * open_pos.size
                    r = (exit_px - open_pos.entry_px) / (open_pos.entry_px - open_pos.stop_px + 1e-9)
                    equity += pnl
                    open_pos.exit_time = when; open_pos.exit_px = exit_px
                    open_pos.pnl = pnl; open_pos.r_mult = r
                    trades.append(open_pos); open_pos = None

            else:  # short
                hit_sl = (nx["High"] >= open_pos.stop_px)
                hit_tp = (nx["Low"]  <= open_pos.tp_px)
                exit_px = None
                when = nt
                if hit_sl and hit_tp:
                    exit_px = open_pos.tp_px if TP_HIT_PRIORITY == "tp_first" else open_pos.stop_px
                elif hit_sl: exit_px = open_pos.stop_px
                elif hit_tp: exit_px = open_pos.tp_px
                if exit_px is not None:
                    # buying to cover
                    buy_after_costs, _ = _apply_costs(exit_px)
                    pnl = (open_pos.entry_px_after_costs - buy_after_costs) * open_pos.size
                    r = (open_pos.entry_px - exit_px) / (open_pos.stop_px - open_pos.entry_px + 1e-9)
                    equity += pnl
                    open_pos.exit_time = when; open_pos.exit_px = exit_px
                    open_pos.pnl = pnl; open_pos.r_mult = r
                    trades.append(open_pos); open_pos = None

        # Entries at next open
        if open_pos is None:
            if row["long_entry"]:
                risk = ATR_MULT_SL * row["R_per_price"]
                stop_px = max(1e-6, row["Close"] - risk)
                tp_px   = row["Close"] + TP_R_MULT * (row["Close"] - stop_px)
                size    = _size_position(equity, RISK_PCT, row["Close"], stop_px)
                if size > 0:
                    entry_px_raw = df.iloc[i+1]["Open"]
                    entry_buy_costed, _ = _apply_costs(entry_px_raw)
                    tr = Trade("long", nt, float(entry_px_raw), float(stop_px), float(tp_px), float(size))
                    tr.entry_px_after_costs = float(entry_buy_costed)
                    open_pos = tr

            elif ALLOW_SHORTS and row["short_entry"]:
                risk = ATR_MULT_SL * row["R_per_price"]
                stop_px = row["Close"] + risk
                tp_px   = row["Close"] - TP_R_MULT * (stop_px - row["Close"])
                size    = _size_position(equity, RISK_PCT, stop_px, row["Close"])
                if size > 0:
                    entry_px_raw = df.iloc[i+1]["Open"]
                    _, entry_sell_costed = _apply_costs(entry_px_raw)
                    tr = Trade("short", nt, float(entry_px_raw), float(stop_px), float(tp_px), float(size))
                    tr.entry_px_after_costs = float(entry_sell_costed)
                    open_pos = tr

    # Close at final bar close if still open
    if open_pos is not None:
        last_close = df["Close"].iloc[-1]
        if open_pos.side == "long":
            _, exit_after_costs = _apply_costs(last_close)
            pnl = (exit_after_costs - open_pos.entry_px_after_costs) * open_pos.size
            r   = (last_close - open_pos.entry_px) / (open_pos.entry_px - open_pos.stop_px + 1e-9)
        else:
            buy_after_costs, _ = _apply_costs(last_close)
            pnl = (open_pos.entry_px_after_costs - buy_after_costs) * open_pos.size
            r   = (open_pos.entry_px - last_close) / (open_pos.stop_px - open_pos.entry_px + 1e-9)
        open_pos.exit_time = df.index[-1]; open_pos.exit_px = float(last_close)
        open_pos.pnl = float(pnl); open_pos.r_mult = float(r)
        trades.append(open_pos); open_pos = None

    # Summaries
    if trades:
        pnl_arr = np.array([t.pnl for t in trades], dtype=float)
        r_arr   = np.array([t.r_mult for t in trades], dtype=float)
        wins    = (pnl_arr > 0).sum()
        ret_sum = pnl_arr.sum()
        winrate = wins / len(trades)
        curve = np.cumsum(pnl_arr) + CAPITAL_PER_TKR
        dd = np.maximum.accumulate(curve) - curve
        maxdd = float(dd.max()) if len(dd) else 0.0
        ret_per_trade = pnl_arr / CAPITAL_PER_TKR
        sharpe = float((ret_per_trade.mean() / (ret_per_trade.std() + 1e-9)) * np.sqrt(252/ max(1,len(ret_per_trade))))
        avg_R = float(r_arr.mean()); med_R = float(np.median(r_arr))
    else:
        ret_sum = 0.0; winrate = 0.0; maxdd = 0.0; sharpe = 0.0; avg_R = 0.0; med_R = 0.0
    summary = {
        "trades": len(trades),
        "profit_total": float(ret_sum),
        "winrate": float(winrate),
        "avg_R": float(avg_R),
        "median_R": float(med_R),
        "max_drawdown": float(maxdd),
        "sharpe_like": float(sharpe),
    }
    return summary, trades

# -----------------------
# 6) RUN
# -----------------------
def run_all():
    np.random.seed(RANDOM_SEED)
    all_summaries = []
    all_trades = []

    for tk in TICKERS:
        print(f"Processing: {tk}")
        df = load_ohlc(tk, START_DATE, END_DATE, INTERVAL)
        df = add_indicators(df)
        df = add_smc(df)
        df = gen_signals(df)

        summary, trades = simulate_ticker(df)
        all_summaries.append({"ticker": tk, **summary})

        for t in trades:
            all_trades.append({
                "ticker": tk,
                "side": t.side,
                "entry_time": t.entry_time,
                "entry_px": t.entry_px,
                "stop_px": t.stop_px,
                "tp_px": t.tp_px,
                "exit_time": t.exit_time,
                "exit_px": t.exit_px,
                "size": t.size,
                "pnl": t.pnl,
                "R_multiple": t.r_mult,
            })

    results_df = pd.DataFrame(all_summaries)
    trades_df  = pd.DataFrame(all_trades)
    results_df.to_csv("smc_backtest_summary.csv", index=False)
    trades_df.to_csv("smc_backtest_trades.csv", index=False)
    print("\n=== Summary by Ticker ===")
    print(results_df.to_string(index=False))
    print("\nSaved: smc_backtest_summary.csv, smc_backtest_trades.csv")
    return results_df, trades_df

if __name__ == "__main__":
    run_all()


[INFO] `smartmoneyconcepts` not found – using fallback SMC detectors (FVG + basic BOS). No module named 'smartmoneyconcepts'
Processing: RELIANCE.NS
Processing: TCS.NS
Processing: INFY.NS
Processing: HDFCBANK.NS

=== Summary by Ticker ===
     ticker  trades  profit_total  winrate  avg_R  median_R  max_drawdown  sharpe_like
RELIANCE.NS      21    9,842.4289   0.5238 0.2456    0.5255   10,753.3824       0.5292
     TCS.NS      12   22,771.1614   0.7500 0.8239    1.3904    5,411.4176       3.2432
    INFY.NS      19   16,110.8529   0.6316 0.3287    0.8960    5,797.2099       1.1086
HDFCBANK.NS      18   13,996.9091   0.6111 0.3694    0.8130   10,604.1042       0.9420

Saved: smc_backtest_summary.csv, smc_backtest_trades.csv
